In [9]:
import os
import gc
import folium 
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import io

from geopandas import GeoDataFrame, GeoSeries, read_file, sjoin
from shapely.geometry import Point, Polygon

In [10]:
geo_path = './geo/geo_data'
sur_path = './Commercial_district_information'

In [11]:
# 파일 프로세싱에 관한 클래스
class file_processing() :
    def __init__(self, sur_path) :
        # 원하는 기간 입력 받아 놓고 기본 폴더내의 파일 리스트 생성
        file_list = os.listdir(sur_path)
        file_list = [i for i in file_list if not os.path.isdir(sur_path+'/'+i)]
        
        want_year = input('년?')
        want_start_q = input('시작 분기?')
        want_end_q = input('끝 분기?')
        
        self.sur_path = sur_path
        self.want_year = want_year
        self.want_start_q = want_start_q
        self.want_end_q = want_end_q
        self.file_list = file_list
    
    # 데이터를 원하는 기간 만큼만 잘라줌
    def filter_period(self, df) :
    
        df = df[df['기준_년_코드'] == int(self.want_year)]
        df = df[df['기준_분기_코드'] >= int(self.want_start_q)]
        df = df[df['기준_분기_코드'] <= int(self.want_end_q)]

        return df
    
    # 영역과 지점간의 매칭 dictionary 생성
    def make_matching_dictionary(self, df_matching_data) :
        dict_matching = {}
        for i in range(len(df_matching_data)) :
            dict_matching[df_matching_data.iloc[i,0]] = df_matching_data.iloc[i,1]
        return dict_matching

    # 하나의 상권에 여러 개 데이터 있을경우 하나로 합쳐줌(평균 처리)
    def merge_same_area_data(self, a) :
        for filename in self.file_list :
            if filename == '서울시우리마을가게상권분석서비스(상권-추정매출)_2020.csv' :
                self.merge_same_area_data_1(filename, a)
            else :
                self.merge_same_area_data_2(filename, a)
            
    # 상권 추정 파일의 경우 컬럼 구성이 달라서 따로 처리
    def merge_same_area_data_1(self, file_name, df_matching_data) :  # 상권 추정매출 전용
        print(file_name, '진입')
        dataframe_existance = False
        except_columns = ['기준_년_코드','기준_분기_코드','상권_구분_코드','상권_구분_코드_명','상권_코드','상권_코드_명','서비스_업종_코드','서비스_업종_코드_명','matching_상권_코드_명']

        df_raw_data = pd.read_csv(self.sur_path+'/'+file_name, index_col=False, encoding='cp949')
        df_raw_data = self.filter_period(df_raw_data)

        dict_matching = self.make_matching_dictionary(df_matching_data[['raw_상권_코드_명','matching_상권명']])

        df_raw_data['matching_상권_코드_명']=df_raw_data['상권_코드_명']
        df_raw_data['matching_상권_코드_명'] = df_raw_data['matching_상권_코드_명'].apply(lambda x:dict_matching.get(x,np.nan))
        
        df_raw_data.to_csv('df_raw_Data.csv', encoding='cp949')

        cnt=0
        for i in  df_raw_data['matching_상권_코드_명'].unique() :                 # unique 한 'matching_상권_코드_명'별로 묶어서 처리함
            df_tmp = df_raw_data[df_raw_data['matching_상권_코드_명']==i]
            if cnt==20:
                print('■',end='')
                cnt=0
            cnt+=1

            for j in df_tmp['기준_분기_코드'].unique() :                          # unique 한 '기준_분기_코드'별로 묶어서 처리함
                df_tmp1 = df_tmp[df_tmp['기준_분기_코드']==j]                     # matching_상권_코드_명','기준_분기_코드'묶은 데이터프레임

                for k in df_tmp1['서비스_업종_코드_명'].unique() :                # unique 한 '서비스_업종_코드_명'별로 묶어서 처리함
                    df_tmp2 = df_tmp1[df_tmp1['서비스_업종_코드_명']==k]          # matching_상권_코드_명','기준_분기_코드'묶은 데이터프레임
                    if len(df_tmp2)>=2 :                                          # 하나의 서비스_업종_코드_명'이 두개 이상 값이 있으면
                        list_tmp = []                                             # 정리된 데이터 프레임에 들어갈 한줄을 위한 임시 리스트

                        for l in df_tmp2.columns :                                # 칼럼 별 순서대로 처리할 것임
                            if l in except_columns :                              # 병합 제외 칼럼 처리
                                list_tmp.append(df_tmp2.iloc[-1].loc[l])          # 해당 칼럼의 마지막 값을 임의로 정해서 병합 할거임
                            else :                                                # 병합 제외 아닌 정상 칼럼이면
                                list_tmp.append(float(df_tmp2.loc[:,l].sum())/len(df_tmp2))  # 칼럼별 합을 데이터 갯수로 나눠서 평균을 내어 임시 리스트에 추가

                        if dataframe_existance :                                  # df_merged 데이터프레임이 이미 있으면
                            df_list = pd.DataFrame(list_tmp).T
                            df_list.columns = list(df_tmp2.columns)
                            df_merged = pd.concat([df_merged,df_list])            # 거기에 그냥 붙이기
                        else :                                                    # df_merged 가 없으면
                            df_merged = pd.DataFrame(list_tmp).T                  # 임시 데이터로 새로 만듬
                            df_merged.columns = list(df_tmp2.columns)
                            dataframe_existance=True                             # 데이터 프레임 만들었다~~
                        del list_tmp

                    else :                                                        # 하나의 서비스_업종_코드_명'이 한개만 값이 있으면    
                        if dataframe_existance :
                            df_merged = pd.concat([df_merged,df_tmp2])
                        else :

                            df_merged = df_tmp2.copy()
                            dataframe_existance=True
        
        if not os.path.exists(sur_path+'/filtered') :
            os.mkdir(sur_path+'/filtered')
        df_merged.to_csv(self.sur_path+'/filtered/'+file_name, encoding='cp949')

    def merge_same_area_data_2(self, file_name, df_matching_data) :     # 상권 추정매출 제외 나머지 파일 처리 용
        print(file_name, '진입')
        dataframe_existance = False
        except_columns = ['기준_년_코드','기준_분기_코드','상권_구분_코드','상권_구분_코드_명',' 상권_구분_코드_명','상권_코드','상권_코드_명','서비스_업종_코드','서비스_업종_코드_명','matching_상권_코드_명']
        
        df_raw_data = pd.read_csv(self.sur_path+'/'+file_name, index_col=False, encoding='cp949')
        df_raw_data = self.filter_period(df_raw_data)

        dict_matching = self.make_matching_dictionary(df_matching_data[['raw_상권_코드_명','matching_상권명']])

        df_raw_data['matching_상권_코드_명'] = df_raw_data['상권_코드_명']
        df_raw_data['matching_상권_코드_명'] = df_raw_data['matching_상권_코드_명'].apply(lambda x:dict_matching.get(x,np.nan))

        cnt=0
        for i in  df_raw_data['matching_상권_코드_명'].unique() :          # unique 한 'matching_상권_코드_명'별로 묶어서 처리함
            df_tmp = df_raw_data[df_raw_data['matching_상권_코드_명']==i]
            if cnt==20:
                print('■',end='')
                cnt=0
            cnt+=1
            for j in df_tmp['기준_분기_코드'].unique() :              # unique 한 '기준_분기_코드'별로 묶어서 처리함
                df_tmp1 = df_tmp[df_tmp['기준_분기_코드']==j]          # matching_상권_코드_명','기준_분기_코드'묶은 데이터프레임

                if len(df_tmp1)>=2 :                                   # 하나의 서비스_업종_코드_명'이 두개 이상 값이 있으면
                    list_tmp = []                                       # 정리된 데이터 프레임에 들어갈 한줄을 위한 임시 리스트

                    for k in df_tmp1.columns :                          # 칼럼 별 순서대로 처리할 것임
                        if k in except_columns :                        # 병합 제외 칼럼 처리
                            list_tmp.append(df_tmp1.iloc[-1].loc[k])    # 해당 칼럼의 마지막 값을 임의로 정해서 병합 할거임
                        else :                                          # 병합 제외 아닌 정상 칼럼이면
                            list_tmp.append(float(df_tmp1.loc[:,k].sum())/len(df_tmp1))  # 칼럼별 합을 데이터 갯수로 나눠서 평균을 내어 임시 리스트에 추가

                    if dataframe_existance :                             # df_merged 데이터프레임이 이미 있으면
                        df_list = pd.DataFrame(list_tmp).T
                        df_list.columns = list(df_tmp1.columns)
                        df_merged = pd.concat([df_merged,df_list])       # 거기에 그냥 붙이기
                    else :                                              # df_merged 가 없으면
                        df_merged = pd.DataFrame(list_tmp).T             # 임시 데이터로 새로 만듬
                        df_merged.columns = list(df_tmp1.columns)
                        dataframe_existance=True                        # 데이터 프레임 만들었다~~
                    del list_tmp

                else :                                                  # 하나의 서비스_업종_코드_명'이 한개만 값이 있으면  
                    if dataframe_existance :
                        df_merged = pd.concat([df_merged,df_tmp1])
                    else :

                        df_merged = df_tmp1.copy()
                        dataframe_existance=True

        if not os.path.exists(sur_path+'/filtered') :
            os.mkdir(sur_path+'/filtered')
        df_merged.to_csv(self.sur_path+'/filtered/'+file_name, encoding='cp949')
        print(file_name+'처리 끝')

# 상권 지점, 상권 지역 데이터 간의 데이터 변환 및 상권별 데이터 처리하는 클래스
class area_info() :
    def __init__(self, path) :
        df = gpd.read_file(path+'/area.zip.geojson')
        df_poly = df[['TRDAR_CD_N','geometry']]
        df_poly.columns=['matching_상권명','geometry']
        print(df_poly)
        
        self.df_poly = df_poly
        
    # 서울 상권 지점 데이터의 5181 좌표를 4326 좌표로 변환 
    def load_seoul_area_data(self) : 
        df =pd.read_csv(geo_path + '/inf/서울시 우리마을가게 상권분석서비스(상권영역).csv', encoding='cp949',index_col = False)
        df = df[['상권_구분_코드_명','상권_코드','상권_코드_명','엑스좌표_값','와이좌표_값']]
        df.columns = ['raw_상권_구분_코드_명','raw_상권_코드','raw_상권_코드_명','엑스좌표_값','와이좌표_값']
        df = GeoDataFrame(df, geometry=gpd.points_from_xy(df['엑스좌표_값'],df['와이좌표_값']),crs=5181)
        df.drop(['엑스좌표_값','와이좌표_값'],axis=1,inplace=True)
        df = df.to_crs(epsg=4326)

        return df

    # 서울 상권 지점 데이터 4326 데이터를 geoDataFrame 으로 변환
    def convert_geo_to_geodataframe(self, df_geo) :
        geo_cul_num = list(df_geo.columns).index('좌표')

        for i in range(len(df_geo)) :
            df_geo['long']=float(df_geo.iloc[i,geo_cul_num].split(',')[1])
            df_geo['lat']=float(df_geo.iloc[i,geo_cul_num].split(',')[0])

        df_geo.reset_index(inplace=True, drop=True)

        # long, lat 데이터로 GeoDataFrame 변환 후 필요 없는 데이터 모두 삭제. 이름, long, lat 만 남김
        df_geo = df_geo[['지명','long','lat']]
        df_geo = GeoDataFrame(df_geo, geometry=gpd.points_from_xy(df_geo['long'],df_geo['lat']))
        df_geo.drop(['long','lat'],axis=1,inplace=True)
        return df_geo
        
    def is_in_area(self, point) :
        point = point[point.geometry.type == 'Point']
        inf_poly = gpd.sjoin(point, self.df_poly, how='inner', op='within')
        return inf_poly
    
# 정리된 확진자 데이터를 불러와서 서울시 데이터만 남김
    def load_infection_data(self) :
        df =pd.read_csv('infection_data.csv', index_col = False)
        df = df[['시','지명','좌표']]
        df = df[df['시']=='서울특별시']
        return df
    

# 정리, 통합된 상권정보에서 확진자가 나오지 않은 상권은 제외 함
def remove_uninfection_area (path,name):
    df_poly_286 = gpd.read_file('df_poly_286.geojson')
    infection_area_name_list = list(df_poly_286['상권명'])
    df = pd.read_csv(path+'/'+name,encoding='cp949')
    for i in range(len(df)-1,-1,-1) :
        if df.iloc[i].loc['matching_상권_코드_명'] not in infection_area_name_list :
            df.drop(index=i, inplace=True)
        else : 
            pass
    df.to_csv(path+'/filtered/filtered/'+name,encoding='cp949')

# 상권 정보 중에 확진자가 발견 되지 않은 지역의 데이터 행을 제거
def remove_uninfection_area (path):
    
    file_list = os.listdir(path+'/filtered')
    file_list = [i for i in file_list if not os.path.isdir(path+'/filtered/'+i)]

    df_poly_286 = gpd.read_file('df_poly_286.geojson')
    infection_area_name_list = list(df_poly_286['상권명'])
    
    
    for i in file_list:
        df = pd.read_csv(path+'/filtered/'+i,encoding='cp949')
        for j in range(len(df)-1,-1,-1) :
            if df.iloc[j].loc['matching_상권_코드_명'] not in infection_area_name_list :
                df.drop(index=j, inplace=True)
            else : 
                pass
        df.to_csv(path+'/filtered/filtered/'+i,encoding='cp949')

    




In [12]:
area = area_info(geo_path)
        
df_infection = area.load_infection_data()
geo_infection = area.convert_geo_to_geodataframe(df_infection)
geo_seoul_area = area.load_seoul_area_data()
a = area.is_in_area(geo_seoul_area)

fp = file_processing(sur_path)

fp.merge_same_area_data(a)

remove_uninfection_area(sur_path) 


         matching_상권명                                           geometry
0               낙성대시장  POLYGON ((126.95719 37.47880, 126.95652 37.477...
1            봉천제일종합시장  POLYGON ((126.94472 37.48094, 126.94404 37.481...
2                도곡시장  POLYGON ((127.05253 37.49702, 127.05192 37.497...
3              강남개포시장  POLYGON ((127.06765 37.48935, 127.06904 37.489...
4              화곡본동시장  POLYGON ((126.84379 37.54250, 126.84272 37.543...
...               ...                                                ...
1489            양재역_3  POLYGON ((127.03268 37.48735, 127.03196 37.487...
1490            양재역_2  POLYGON ((127.03805 37.48366, 127.03748 37.483...
1491     서울 관악구 신림역_1  POLYGON ((126.92889 37.48178, 126.92910 37.481...
1492     서울 관악구 신림역_2  POLYGON ((126.93000 37.48128, 126.92969 37.484...
1493  서울 서초구 남부터미널역_1  POLYGON ((127.01975 37.48412, 127.01957 37.483...

[1494 rows x 2 columns]


C:\Users\story\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


년?2020
시작 분기?1
끝 분기?2
서울시 우리마을가게 상권분석서비스(상권-생활인구).csv 진입
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■서울시 우리마을가게 상권분석서비스(상권-생활인구).csv처리 끝
서울시 우리마을가게 상권분석서비스(상권-아파트).csv 진입
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■서울시 우리마을가게 상권분석서비스(상권-아파트).csv처리 끝
서울시 우리마을가게 상권분석서비스(상권-직장인구).csv 진입
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■서울시 우리마을가게 상권분석서비스(상권-직장인구).csv처리 끝
서울시 우리마을가게 상권분석서비스(상권-집객시설).csv 진입
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■서울시 우리마을가게 상권분석서비스(상권-집객시설).csv처리 끝
서울시우리마을가게상권분석서비스(상권-추정매출)_2020.csv 진입
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■